In [1]:
import numpy as np
from ConfigGeneration import pyg_graph_gen
from GenericAlgorithm import GenericAlgorithm
from SchNet import RBFLayer,InteractionBlock,cfconv,FilterGeneratorBlock,Schnet
import torch
from torch_geometric.data import DataLoader
import random

In [2]:
random.seed(0)

In [3]:
x_period=3
y_period=3
z_period=3
variable_bounds=list()
variable_types=list()
for i in range(2*x_period*y_period*z_period):
    variable_bounds.append([0,2])
    variable_types.append('int')
for i in range(6*x_period*y_period*z_period):
    variable_bounds.append([-0.02,0.02])
    variable_types.append('float')
variable_bounds=np.array(variable_bounds)

In [4]:
model_1=torch.load('../model_blending/best_model_1.pkl',map_location='cpu')
model_2=torch.load('../model_blending/best_model_2.pkl',map_location='cpu')
model_3=torch.load('../model_blending/best_model_3.pkl',map_location='cpu')

In [5]:
def gen_function(models,preprocessing_func,*preprocessing_func_params):
    model_num=len(models)
    def mean_function(atom_configs):
        pred_energies=list()
        for i in range(model_num):
            pred_energies.append(list())
        atom_graphs=list()
        for i in range(len(atom_configs)):
            atom_graphs.append(preprocessing_func(atom_configs[i],*preprocessing_func_params))
        atom_graphs_loader=DataLoader(atom_graphs,batch_size=10)
        for temp in atom_graphs_loader:
            for i in range(model_num):
                pred_energy=models[i](temp)*100
                pred_energies[i].append(pred_energy.cpu().detach().numpy())
        for i in range(model_num):
            pred_energies[i]=np.concatenate(pred_energies[i],axis=0)
        return np.mean(np.squeeze(np.array(pred_energies)),axis=0)
    return mean_function

In [6]:
opt_func=gen_function([model_1,model_2,model_3],pyg_graph_gen,x_period,y_period,z_period)
ga=GenericAlgorithm(opt_func,variable_bounds,variable_types=variable_types,opt_target='min',population_size=1000)

In [7]:
ga.evolution(num_iterations=100)

iteration:  0  time=  0:02:27.748022  min/max value=  -464.3235
iteration:  1  time=  0:02:47.733978  min/max value=  -479.1287
iteration:  2  time=  0:02:41.917088  min/max value=  -491.52243
iteration:  3  time=  0:02:38.835332  min/max value=  -496.74887
iteration:  4  time=  0:02:45.282092  min/max value=  -501.62387
iteration:  5  time=  0:02:45.178369  min/max value=  -501.6344
iteration:  6  time=  0:02:44.615873  min/max value=  -501.64145
iteration:  7  time=  0:02:43.050064  min/max value=  -501.64417
iteration:  8  time=  0:02:49.087917  min/max value=  -501.64566
iteration:  9  time=  0:02:46.105889  min/max value=  -501.6425
iteration:  10  time=  0:02:49.617500  min/max value=  -501.64804
iteration:  11  time=  0:02:45.294060  min/max value=  -501.65796
iteration:  12  time=  0:02:51.910370  min/max value=  -501.65945
iteration:  13  time=  0:02:42.962295  min/max value=  -501.66382
iteration:  14  time=  0:02:44.122822  min/max value=  -501.65552
iteration:  15  time=  0

KeyboardInterrupt: 

In [8]:
temp_a,temp_b=ga.get_result()

In [10]:
for i in range(len(temp_a)):
    for j in range(len(temp_a[i])):
        print(temp_a[i,j],',',end='')
    print(temp_b[i])

2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,1 ,1 ,1 ,2 ,1 ,2 ,1 ,2 ,1 ,1 ,2 ,1 ,2 ,0 ,2 ,0 ,1 ,2 ,1 ,2 ,1 ,2 ,0 ,2 ,1 ,2 ,0 ,1 ,2 ,0 ,0 ,1 ,1 ,0.01610723438901292 ,-0.007546418588862416 ,-0.010220289081185881 ,0.013594500466606391 ,0.002411032332295133 ,-0.0016295334381054467 ,-0.0153341703452075 ,-0.013445260676784813 ,0.014034251075502099 ,-0.006732310425917282 ,0.013217660204655039 ,0.014686101116710589 ,-0.010040367808202486 ,0.0034706634529192842 ,-0.0015975512523421567 ,-0.012865657718531237 ,-0.0178615964682618 ,-0.0077612868788781725 ,-0.015647926804866593 ,0.008474423723737846 ,-0.004091403239009597 ,0.017217949910179896 ,0.010081844345096735 ,-0.011774726882101412 ,0.003780905842969161 ,-0.01413415495202274 ,-0.01764091895697702 ,-0.010929826067409625 ,0.005227533852310104 ,0.0016960381852167694 ,-0.015532805325780556 ,0.010174163143861954 ,0.0025953799260719536 ,0.0005498314985784881 ,-0.015470373281824577 ,-0.01212262636258095 ,0.009416839661542865 ,-0.00

2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,0 ,0 ,1 ,2 ,0 ,0 ,2 ,0 ,2 ,0 ,2 ,1 ,1 ,0 ,1 ,1 ,2 ,0 ,1 ,1 ,0 ,1 ,1 ,1 ,2 ,0 ,1 ,0 ,2 ,2 ,0 ,1 ,0 ,2 ,1 ,2 ,0.007615470131959786 ,-0.006079397544263036 ,0.012161415831743964 ,-0.0027675122039295154 ,-0.0073321776413718 ,-0.006444438076281979 ,0.008167043033344715 ,0.015104613770980328 ,0.011992072630672184 ,-0.019239395934568543 ,-0.006012057364338009 ,-0.00784926481679073 ,0.010962098973725624 ,-0.00900333207128014 ,-0.011250601242389071 ,-0.011080691163308138 ,-0.009823702725685121 ,0.006560228640139915 ,-0.01903807120646883 ,-0.006553463795028459 ,0.013064939999305369 ,0.011926276185461108 ,0.013945282552692461 ,0.009004839488093573 ,-0.005563594890597696 ,-0.016011640783083293 ,0.003087548943267352 ,0.010988259741351293 ,-0.0014413913529364927 ,-0.009700324674739563 ,-0.011407281366404596 ,0.0022419825907922207 ,-0.0029923933126198773 ,0.0002372079063075426 ,0.0035801158550000448 ,0.010652945409329203 ,0.012157713336303135 ,0.01

2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,0 ,0 ,0 ,1 ,1 ,2 ,2 ,0 ,2 ,0 ,0 ,1 ,1 ,1 ,1 ,1 ,0 ,2 ,0 ,2 ,1 ,2 ,2 ,0 ,0 ,0 ,2 ,0 ,1 ,2 ,2 ,0 ,2 ,2 ,1 ,0 ,0.010850655103156144 ,-0.007546418588862416 ,-0.010220289081185881 ,0.013594500466606391 ,0.003133153230039293 ,0.0005557387616340302 ,-0.008633468807972165 ,0.0027960055140766257 ,-0.0009643746662671669 ,0.00654425232425938 ,-0.006881783110046507 ,-0.016892496065343286 ,0.01848570623797856 ,0.004114163310165099 ,0.0006363627455660166 ,-0.005524169642971359 ,0.006375940856573142 ,0.01737450274736495 ,-0.01907084831616082 ,0.0038717923884652415 ,-0.019208512627814617 ,-0.017790941544315878 ,-0.013219632801092174 ,-0.002751669967158428 ,0.014291635750248905 ,0.014828597491894222 ,0.001755562004205631 ,0.013928551290463834 ,0.01841154611842839 ,-0.013576106799489295 ,-0.005678259853175018 ,0.011600149417660088 ,-0.005034542505633231 ,-0.010728411880571102 ,0.0024831592506370723 ,-0.013558594623112081 ,-0.005005527581544182 ,-0.01

2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,0 ,0 ,0 ,1 ,1 ,2 ,2 ,0 ,0 ,1 ,0 ,2 ,2 ,2 ,2 ,1 ,1 ,2 ,0 ,1 ,1 ,2 ,0 ,0 ,2 ,2 ,0 ,1 ,0 ,1 ,0 ,1 ,1 ,1 ,1 ,-0.019080978174091483 ,-0.0054168805640585835 ,0.0075614140292365635 ,-0.00033854298354415654 ,0.0027653865083251718 ,0.016226866963625585 ,0.00040811958373171767 ,-0.008889947840032716 ,-0.013025314351832278 ,0.014069598923860043 ,0.009924855555939499 ,0.014061681817378279 ,-0.01975657189047126 ,-0.015550498420745953 ,-0.013212793450016407 ,-0.016346731567237913 ,0.004688729920489618 ,-0.00882983606656846 ,0.01975007154901977 ,-0.003603582687589587 ,-0.006882081179744129 ,0.015683103962184957 ,-0.013219632801092174 ,-0.00941274363386944 ,0.010625401311802445 ,-0.009747307091819203 ,-0.004671215314521774 ,0.004987392672904739 ,0.01532885575933263 ,-0.013141458075815038 ,0.010678781469069039 ,0.013716873032977698 ,-0.017058976398782035 ,-0.006782564017131407 ,-0.015470373281824577 ,0.0023488513606512336 ,0.003186943217000051 ,0

2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,0 ,0 ,0 ,1 ,1 ,1 ,2 ,0 ,1 ,0 ,0 ,1 ,1 ,1 ,1 ,1 ,0 ,2 ,0 ,0 ,1 ,2 ,1 ,0 ,0 ,0 ,2 ,0 ,1 ,2 ,2 ,0 ,2 ,2 ,1 ,0 ,0.015650343898894834 ,0.0028854111703960766 ,-0.010220289081185881 ,-0.013554477665367354 ,0.003133153230039293 ,0.0005557387616340302 ,-0.008633468807972165 ,0.0027960055140766257 ,-0.0014103372477229578 ,0.010301237553762797 ,-0.006881783110046507 ,-0.016892496065343286 ,0.01848570623797856 ,-0.0011019122632143843 ,0.0006363627455660166 ,-0.014306460978592108 ,0.006809818807303688 ,0.01737450274736495 ,-0.01907084831616082 ,0.0008572214540405465 ,-0.0018164321964684663 ,0.006996549121716215 ,0.008627402671704986 ,-0.004166195541959298 ,-0.00048265669179718 ,0.00043623872668880834 ,0.018337693241342556 ,-0.0006055731233153491 ,0.003082302681919242 ,-0.0017085808093248098 ,0.018293523675825003 ,0.0056052842296763945 ,-0.016294955623420707 ,-0.001927051733859729 ,-0.015470373281824577 ,-0.016686941353954314 ,0.01628431852418052

...

2 ,2 ,2 ,2 ,2 ,1 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,1 ,1 ,2 ,2 ,2 ,0 ,1 ,0 ,1 ,1 ,0 ,2 ,0 ,2 ,1 ,1 ,0 ,0 ,2 ,1 ,1 ,2 ,1 ,2 ,2 ,0 ,2 ,1 ,2 ,2 ,0 ,2 ,2 ,1 ,2 ,1 ,0 ,0.00022464537528743903 ,0.0009593133262976138 ,0.01586897076017402 ,0.017221788076392467 ,0.017001982747593774 ,0.011571020851192031 ,-0.007685442947525444 ,-0.005807426408947417 ,-0.014145292437368936 ,0.018576708589527053 ,0.010396700392514592 ,0.006515297703107156 ,-0.017621444133750074 ,0.0075528866845417965 ,0.013842886040807943 ,0.013370724395526144 ,0.01674816281304362 ,-0.006187938732906817 ,-0.018962732552359137 ,0.0038674242318352432 ,-0.019542181185841407 ,0.006996549121716215 ,0.003876108868685186 ,-0.0063402310006956866 ,0.002193358333039231 ,0.005792135820159788 ,-0.002814375960549332 ,-0.011954180090746364 ,-0.018692889918419175 ,-0.004036089260724074 ,0.015319799643069194 ,0.009526343582854743 ,-0.0011114684363644593 ,-0.0191185246408022 ,-0.012401563204725248 ,-0.005322420722904923 ,-0.0023446711391331213 ,-0.

2 ,2 ,2 ,2 ,2 ,1 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,1 ,2 ,1 ,2 ,2 ,2 ,0 ,0 ,0 ,2 ,0 ,2 ,2 ,2 ,0 ,0 ,1 ,1 ,1 ,2 ,1 ,1 ,2 ,1 ,0 ,0 ,2 ,0 ,0 ,0 ,1 ,2 ,1 ,0 ,2 ,1 ,2 ,2 ,-0.014024470817094997 ,0.0021490042895580923 ,0.013568692239841692 ,0.015031416811673037 ,-0.008291199209105394 ,-0.019122032070219716 ,-0.015030072809179935 ,-0.0009273833549189453 ,-0.009707427461311164 ,0.018084592986071592 ,-0.01446243735399408 ,0.01985873385277757 ,-0.0011141725669970015 ,-0.005072385765720541 ,-0.008983473129806971 ,0.004134424367813243 ,-0.004611070470172148 ,0.007647072487124009 ,-0.011460576454521433 ,0.019475715663377784 ,-0.002575402868686242 ,-0.00023139123421319838 ,-0.01784615084239769 ,-0.01693928854379316 ,-0.0025827366324354013 ,-0.009747307091819203 ,0.014109966720273062 ,0.00652816650050304 ,0.008404331943561123 ,0.016766401087251425 ,0.01586376408781272 ,0.0019345493075524703 ,0.018843725384825354 ,-0.010860430544396648 ,0.0045977326665658826 ,-0.01781038304441364 ,-0.004335177449282218 ,0.

2 ,2 ,2 ,2 ,2 ,1 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,1 ,2 ,2 ,2 ,2 ,1 ,0 ,2 ,0 ,1 ,2 ,2 ,2 ,0 ,1 ,2 ,0 ,0 ,0 ,0 ,2 ,0 ,0 ,1 ,2 ,0 ,1 ,2 ,1 ,2 ,2 ,2 ,2 ,2 ,1 ,1 ,0 ,1 ,2 ,2 ,2 ,0.012660670537020993 ,-0.014570013059137285 ,0.013375910651469269 ,0.01834662143220352 ,0.016811717985689086 ,-0.014255033317115095 ,0.008763602599529324 ,0.019415541543828952 ,-0.01500728605072256 ,0.018576708589527053 ,0.012558579825644064 ,-0.012739712588372459 ,-0.0047179541215506896 ,-0.004868671769891945 ,0.01099437099076813 ,-0.005703793816544204 ,-0.0036183936987278335 ,-0.00882983606656846 ,0.01788891777541694 ,0.010886921149910616 ,-0.019470542634669755 ,-0.00805079290633234 ,-0.013219632801092174 ,0.0009121828986340362 ,-0.017180224985176344 ,-0.009747307091819203 ,-0.011848465197194843 ,-0.01069216240385705 ,-0.015143172300811282 ,0.00038398447933734453 ,-0.0003130128602454804 ,0.015013239002210497 ,0.0040317814694656486 ,-0.01605909837879963 ,0.012729539575709322 ,-0.0026856573030508146 ,-0.018002192688897748 ,-0.

2 ,1 ,1 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,0 ,1 ,0 ,1 ,2 ,2 ,1 ,1 ,0 ,2 ,0 ,1 ,1 ,1 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,2 ,1 ,0 ,0 ,2 ,1 ,1 ,2 ,0 ,1 ,2 ,2 ,2 ,-0.004516998340743683 ,0.015318862635454056 ,0.013015800201066349 ,-0.0037698123778924578 ,-0.0014736193037073117 ,0.015114901094481676 ,-0.013429605985387675 ,0.00980950453300198 ,-0.01750459721095362 ,0.014184850700751633 ,-0.009171535812808544 ,0.013630410880973113 ,0.012708870038852748 ,0.008329878605331357 ,0.010012162030786684 ,-0.011153625741214849 ,0.0185484432714401 ,-0.006260977380613335 ,0.0019107450283022825 ,0.018924783914095206 ,0.0041575285245039235 ,-0.019497419843022126 ,0.002466930070781293 ,-0.01668078960390559 ,0.00545837068248534 ,0.019200802089242613 ,0.003087548943267352 ,0.019972866149177062 ,-0.012956403424637126 ,-0.003481605993075204 ,-0.002396838021574744 ,-0.010124575243798116 ,0.011489962570383112 ,0.008045548391765932 ,0.014861277040116489 ,0.006323617354351985 ,-0.0053814094734253096 ,0.01667817

1 ,2 ,2 ,2 ,1 ,2 ,2 ,2 ,2 ,2 ,1 ,2 ,2 ,2 ,2 ,2 ,2 ,2 ,0 ,0 ,2 ,2 ,1 ,2 ,2 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,0 ,1 ,1 ,1 ,0 ,0 ,2 ,2 ,2 ,0 ,1 ,0 ,2 ,1 ,1 ,2 ,2 ,0 ,0 ,2 ,0.013293292738572073 ,-0.014828781472815203 ,-0.014114001120393977 ,0.01711896585209927 ,-0.019345669829003292 ,0.014243737119952717 ,0.013102625757044437 ,0.0012181526821902974 ,0.008532429251432813 ,-0.011092521253014258 ,0.0053642790161296805 ,-0.013335235762790464 ,0.01157335643816643 ,0.009013567415365582 ,-0.016469719608966364 ,-0.004042134597009094 ,-0.009823702725685121 ,-0.0004788173846512582 ,-0.018524912209889104 ,0.016301753485009684 ,-0.008839916355123201 ,0.013422795655071425 ,-0.017971127134037562 ,0.0032883965949511568 ,0.006400604848727155 ,-0.009747307091819203 ,0.003087548943267352 ,-0.012983117108416336 ,-0.015143172300811282 ,0.008470031096220368 ,-0.018366349121272822 ,-0.009812865604923733 ,-0.0041718975916376876 ,-0.012565467442889028 ,-0.014714279123610164 ,-0.005322420722904923 ,0.00024477098525962915